In [1]:
# !pip3 install medpy -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip3 install nibabel -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip3 install pynrrd -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# cd dataloaders
# !python3 la_heart_processing.py

In [ ]:
!cd /home/cyagen/tyler/CTM/UA-MT/code
!python3 train_VNet_CTM.py --batch_size 8 --gpu "4,5"  #,5,6,7"

Namespace(base_lr=0.01, batch_size=2, deterministic=1, exp='vnet_supervisedonly_dp', gpu='5', max_iterations=6000, root_path='/home/cyagen/tyler/CTM/UA-MT/data/CTM_dataset/Segmented', seed=1337)
total 140 samples
total 36 samples
70 itertations per epoch
  0%|                                          | 0/86 [00:00<?, ?it/s]

dice score (1-dice_loss): 0.691
dice score (1-dice_loss): 0.174
dice score (1-dice_loss): 0.030
dicetotal:2.105
iteration 1 : loss : 1.527344, loss_seg : 0.949742, loss_seg_dice : 2.104946


dice score (1-dice_loss): 0.608
dice score (1-dice_loss): 0.113
dice score (1-dice_loss): 0.173
dicetotal:2.106
iteration 2 : loss : 1.623864, loss_seg : 1.141877, loss_seg_dice : 2.105851


dice score (1-dice_loss): 0.685
dice score (1-dice_loss): 0.301
dice score (1-dice_loss): 0.021
dicetotal:1.992
iteration 3 : loss : 1.514538, loss_seg : 1.036799, loss_seg_dice : 1.992277


dice score (1-dice_loss): 0.704
dice score (1-dice_loss): 0.416
dice score (1-dice_loss): 0.018
dice

iteration 42 : loss : 1.108879, loss_seg : 0.696684, loss_seg_dice : 1.521075


dice score (1-dice_loss): 0.917
dice score (1-dice_loss): 0.629
dice score (1-dice_loss): 0.024
dicetotal:1.430
iteration 43 : loss : 1.057751, loss_seg : 0.685656, loss_seg_dice : 1.429847


dice score (1-dice_loss): 0.948
dice score (1-dice_loss): 0.598
dice score (1-dice_loss): 0.137
dicetotal:1.318
iteration 44 : loss : 0.967170, loss_seg : 0.616814, loss_seg_dice : 1.317527


dice score (1-dice_loss): 0.927
dice score (1-dice_loss): 0.622
dice score (1-dice_loss): 0.118
dicetotal:1.333
iteration 45 : loss : 0.964570, loss_seg : 0.596200, loss_seg_dice : 1.332941


dice score (1-dice_loss): 0.955
dice score (1-dice_loss): 0.815
dice score (1-dice_loss): 0.003
dicetotal:1.227
iteration 46 : loss : 0.786812, loss_seg : 0.346321, loss_seg_dice : 1.227304


dice score (1-dice_loss): 0.935
dice score (1-dice_loss): 0.340
dice score (1-dice_loss): 0.122
dicetotal:1.603
iteration 47 : loss : 1.106725, loss_seg



dice score (1-dice_loss): 0.932
dice score (1-dice_loss): 0.672
dice score (1-dice_loss): 0.123
dicetotal:1.273
iteration 85 : loss : 0.992967, loss_seg : 0.712795, loss_seg_dice : 1.273139


dice score (1-dice_loss): 0.906
dice score (1-dice_loss): 0.360
dice score (1-dice_loss): 0.081
dicetotal:1.653
iteration 86 : loss : 1.143762, loss_seg : 0.634715, loss_seg_dice : 1.652809


dice score (1-dice_loss): 0.927
dice score (1-dice_loss): 0.354
dice score (1-dice_loss): 0.044
dicetotal:1.676
iteration 87 : loss : 1.027358, loss_seg : 0.379123, loss_seg_dice : 1.675593


dice score (1-dice_loss): 0.902
dice score (1-dice_loss): 0.452
dice score (1-dice_loss): 0.000
dicetotal:1.646
iteration 88 : loss : 1.098092, loss_seg : 0.550018, loss_seg_dice : 1.646166


dice score (1-dice_loss): 0.905
dice score (1-dice_loss): 0.403
dice score (1-dice_loss): 0.086
dicetotal:1.606
iteration 89 : loss : 1.075168, loss_seg : 0.544134, loss_seg_dice : 1.606202


dice score (1-dice_loss): 0.946
dice s

iteration 127 : loss : 0.790440, loss_seg : 0.309715, loss_seg_dice : 1.271165


dice score (1-dice_loss): 0.959
dice score (1-dice_loss): 0.593
dice score (1-dice_loss): 0.023
dicetotal:1.424
iteration 128 : loss : 0.834749, loss_seg : 0.245005, loss_seg_dice : 1.424493


dice score (1-dice_loss): 0.946
dice score (1-dice_loss): 0.674
dice score (1-dice_loss): 0.100
dicetotal:1.279
iteration 129 : loss : 0.898422, loss_seg : 0.517779, loss_seg_dice : 1.279065


dice score (1-dice_loss): 0.941
dice score (1-dice_loss): 0.699
dice score (1-dice_loss): 0.000
dicetotal:1.360
iteration 130 : loss : 0.959385, loss_seg : 0.558858, loss_seg_dice : 1.359912


dice score (1-dice_loss): 0.954
dice score (1-dice_loss): 0.459
dice score (1-dice_loss): 0.072
dicetotal:1.515
iteration 131 : loss : 0.913405, loss_seg : 0.311635, loss_seg_dice : 1.515176


dice score (1-dice_loss): 0.874
dice score (1-dice_loss): 0.300
dice score (1-dice_loss): 0.044
dicetotal:1.782
iteration 132 : loss : 1.893607, lo



dice score (1-dice_loss): 0.968
dice score (1-dice_loss): 0.501
dice score (1-dice_loss): 0.173
dicetotal:1.358
iteration 170 : loss : 0.831076, loss_seg : 0.303699, loss_seg_dice : 1.358453


dice score (1-dice_loss): 0.941
dice score (1-dice_loss): 0.501
dice score (1-dice_loss): 0.204
dicetotal:1.353
iteration 171 : loss : 0.977886, loss_seg : 0.602616, loss_seg_dice : 1.353155


dice score (1-dice_loss): 0.962
dice score (1-dice_loss): 0.737
dice score (1-dice_loss): 0.164
dicetotal:1.137
iteration 172 : loss : 0.820315, loss_seg : 0.503555, loss_seg_dice : 1.137075


dice score (1-dice_loss): 0.943
dice score (1-dice_loss): 0.491
dice score (1-dice_loss): 0.083
dicetotal:1.484
iteration 173 : loss : 0.940509, loss_seg : 0.397313, loss_seg_dice : 1.483704


dice score (1-dice_loss): 0.977
dice score (1-dice_loss): 0.701
dice score (1-dice_loss): 0.091
dicetotal:1.232
iteration 174 : loss : 0.761889, loss_seg : 0.291880, loss_seg_dice : 1.231898


dice score (1-dice_loss): 0.974
d

...



dice score (1-dice_loss): 0.967
dice score (1-dice_loss): 0.611
dice score (1-dice_loss): 0.106
dicetotal:1.316
iteration 596 : loss : 0.803156, loss_seg : 0.290468, loss_seg_dice : 1.315844


dice score (1-dice_loss): 0.955
dice score (1-dice_loss): 0.711
dice score (1-dice_loss): 0.081
dicetotal:1.253
iteration 597 : loss : 0.909921, loss_seg : 0.566396, loss_seg_dice : 1.253447


dice score (1-dice_loss): 0.974
dice score (1-dice_loss): 0.325
dice score (1-dice_loss): 0.000
dicetotal:1.700
iteration 598 : loss : 0.933732, loss_seg : 0.167046, loss_seg_dice : 1.700417


dice score (1-dice_loss): 0.952
dice score (1-dice_loss): 0.735
dice score (1-dice_loss): 0.121
dicetotal:1.192
iteration 599 : loss : 0.994234, loss_seg : 0.796087, loss_seg_dice : 1.192382


dice score (1-dice_loss): 0.983
dice score (1-dice_loss): 0.834
dice score (1-dice_loss): 0.042
dicetotal:1.142
iteration 600 : loss : 0.658236, loss_seg : 0.174804, loss_seg_dice : 1.141669


dice score (1-dice_loss): 0.962
d

iteration 638 : loss : 0.517460, loss_seg : 0.213724, loss_seg_dice : 0.821195


dice score (1-dice_loss): 0.977
dice score (1-dice_loss): 0.675
dice score (1-dice_loss): 0.153
dicetotal:1.195
iteration 639 : loss : 0.697566, loss_seg : 0.200281, loss_seg_dice : 1.194851


dice score (1-dice_loss): 0.951
dice score (1-dice_loss): 0.455
dice score (1-dice_loss): 0.000
dicetotal:1.594
iteration 640 : loss : 0.994264, loss_seg : 0.394532, loss_seg_dice : 1.593996


dice score (1-dice_loss): 0.972
dice score (1-dice_loss): 0.018
dice score (1-dice_loss): 0.014
dicetotal:1.996
iteration 641 : loss : 1.686734, loss_seg : 1.377269, loss_seg_dice : 1.996200


dice score (1-dice_loss): 0.977
dice score (1-dice_loss): 0.538
dice score (1-dice_loss): 0.050
dicetotal:1.435
iteration 642 : loss : 0.854455, loss_seg : 0.273930, loss_seg_dice : 1.434981


dice score (1-dice_loss): 0.977
dice score (1-dice_loss): 0.619
dice score (1-dice_loss): 0.037
dicetotal:1.367
iteration 643 : loss : 0.855585, lo

iteration 681 : loss : 0.464933, loss_seg : 0.446270, loss_seg_dice : 0.483596


dice score (1-dice_loss): 0.972
dice score (1-dice_loss): 0.760
dice score (1-dice_loss): 0.780
dicetotal:0.489
iteration 682 : loss : 0.454355, loss_seg : 0.420038, loss_seg_dice : 0.488672


dice score (1-dice_loss): 0.985
dice score (1-dice_loss): 0.783
dice score (1-dice_loss): 0.256
dicetotal:0.977
iteration 683 : loss : 0.607974, loss_seg : 0.238942, loss_seg_dice : 0.977006


dice score (1-dice_loss): 0.960
dice score (1-dice_loss): 0.314
dice score (1-dice_loss): 0.000
dicetotal:1.726
iteration 684 : loss : 1.044875, loss_seg : 0.363554, loss_seg_dice : 1.726196


dice score (1-dice_loss): 0.969
dice score (1-dice_loss): 0.483
dice score (1-dice_loss): 0.161
dicetotal:1.387
iteration 685 : loss : 0.817635, loss_seg : 0.247838, loss_seg_dice : 1.387433


dice score (1-dice_loss): 0.972
dice score (1-dice_loss): 0.780
dice score (1-dice_loss): 0.546
dicetotal:0.702
iteration 686 : loss : 0.530230, lo



dice score (1-dice_loss): 0.941
dice score (1-dice_loss): 0.312
dice score (1-dice_loss): 0.258
dicetotal:1.489
iteration 724 : loss : 1.107304, loss_seg : 0.725575, loss_seg_dice : 1.489034


dice score (1-dice_loss): 0.966
dice score (1-dice_loss): 0.714
dice score (1-dice_loss): 0.005
dicetotal:1.316
iteration 725 : loss : 0.824375, loss_seg : 0.332972, loss_seg_dice : 1.315779


dice score (1-dice_loss): 0.955
dice score (1-dice_loss): 0.587
dice score (1-dice_loss): 0.534
dicetotal:0.924
iteration 726 : loss : 0.703746, loss_seg : 0.483007, loss_seg_dice : 0.924486


dice score (1-dice_loss): 0.981
dice score (1-dice_loss): 0.779
dice score (1-dice_loss): 0.560
dicetotal:0.680
iteration 727 : loss : 0.474580, loss_seg : 0.269294, loss_seg_dice : 0.679865


dice score (1-dice_loss): 0.978
dice score (1-dice_loss): 0.715
dice score (1-dice_loss): 0.102
dicetotal:1.205
iteration 728 : loss : 0.763673, loss_seg : 0.322295, loss_seg_dice : 1.205050


dice score (1-dice_loss): 0.968
d

iteration 766 : loss : 0.994436, loss_seg : 0.731381, loss_seg_dice : 1.257491


dice score (1-dice_loss): 0.958
dice score (1-dice_loss): 0.494
dice score (1-dice_loss): 0.265
dicetotal:1.283
iteration 767 : loss : 1.083940, loss_seg : 0.884715, loss_seg_dice : 1.283166


dice score (1-dice_loss): 0.986
dice score (1-dice_loss): 0.783
dice score (1-dice_loss): 0.724
dicetotal:0.507
iteration 768 : loss : 0.351344, loss_seg : 0.195288, loss_seg_dice : 0.507401


dice score (1-dice_loss): 0.969
dice score (1-dice_loss): 0.479
dice score (1-dice_loss): 0.000
dicetotal:1.551
iteration 769 : loss : 0.890520, loss_seg : 0.229803, loss_seg_dice : 1.551236


dice score (1-dice_loss): 0.978
dice score (1-dice_loss): 0.700
dice score (1-dice_loss): 0.127
dicetotal:1.196
iteration 770 : loss : 0.774910, loss_seg : 0.354201, loss_seg_dice : 1.195619
 13%|###9                           | 11/86 [10:42<1:11:01, 56.82s/it]

dice score (1-dice_loss): 0.962
dice score (1-dice_loss): 0.579
dice score (1

In [ ]:
!python3 test_VNet_CTM.py --gpu '4,5,6,7'

In [ ]:
1658899